### In this tutorial, we show how to modify neural parameters prior to network simulation

## Step 1: Import necessary packages and modules

In [ ]:
# This line allows you to display matplotlib plots within the Jupyter Notebook
%matplotlib inline

# Import External packages 

import os
import numpy as np
import matplotlib.pyplot as plt

# Configure the working directory (Important: This should be set to home directory of 'dynworm' folder)

default_dir = os.path.dirname(os.getcwd())
os.chdir(default_dir)

# Import Main module

import dynworm as dw

## Step 2: Modify neural parameters 

### There are two ways you can modify network parameters: 
### 1) Modifying the physiological neural parameters
### 2) Modifying the connectome

## 2.1: Modifying the physiological constants 

### You can modify the physiological constants by providing your own custom dictionary of neural paramters
### For the description of each parameter, see dynworm.neural_params.py
### Custom dictionary should include all the parameters present in dw.neural_params.pA_unit_baseline
### We use the following custom neural parameter dictionary for this tutorial which modifies cell capacitance (C) and and leakage potential (Ec) 

custom_neural_params = {

    "N" : 279, 
    "Gc" : 0.1,
    "C" : 0.01, (0.015 --> 0.01)
    "ggap" : 1.0,
    "gsyn" : 1.0,
    "Ec" : -45.0, (-35 --> -45)
    "E_rev": -48.0, 
    "ar" : 1.0/1.5,
    "ad" : 5.0/1.5,
    "B" : 0.125,
    "rate" : 0.025,
    "offset" : 0.15,
    "iext" : 10.,
    "nonlinear_AWA": False,
    "nonlinear_AVL": False

    }
    
### The custom dictionary is then provided to dynworm.network_sim.initialize_params_neural() 

## 2.2: Modifying the connectivity matrices

### You can modify the connectivity matrices before initializing the connectomes by providing a following dictionary consisting of new connectomes to dynworm.network_sim.initialize_connectivity()

modified_connectomes = 
            {

            "gap": new gap connectome with dimension (279 * 279),
            "syn": new synaptic connectome (279 * 279),
            "directionality": new boolean matrix for inhibitory neurons (279, 279), 
            where 0 = excitatory, 1 = inhibitory, columns: from, rows: to

            }
            
### Note that you can modify the connectivity matrices via "ablating" neurons as well through ablation_mask

### In this tutorial, we will modify physiological paramters + ablate neurons on the base connectomes

In [ ]:
# Defeine custom dictionary consisting of all the necessary physiological constants

custom_neural_params = {

    "N" : 279, 
    "Gc" : 0.1,
    "C" : 0.01, 
    "ggap" : 1.0,
    "gsyn" : 1.0,
    "Ec" : -45.0, 
    "E_rev": -48.0, 
    "ar" : 1.0/1.5,
    "ad" : 5.0/1.5,
    "B" : 0.125,
    "rate" : 0.025,
    "offset" : 0.15,
    "iext" : 10.,
    "nonlinear_AWA": False,
    "nonlinear_AVL": False

    }

### Initialize neural paramters and connectivity matrices

In [ ]:
dw.network_sim.initialize_params_neural(custom_neural_params)
dw.network_sim.initialize_connectivity()

In [ ]:
# Use dw.utils.neuron_names_2_inds to find the indices of the neurons

print(dw.utils.neuron_names_2_inds(['PVCL']), dw.utils.neuron_names_2_inds(['PVCR']))

### Secondary modification of base connectomes - We ablate PVCL and PVCR, which are known as driver interneurons for forward crawling motion

In [ ]:
ablation_mask = np.ones(dw.network_sim.params_obj_neural['N'], dtype = 'bool')
ablation_mask[261] = False
ablation_mask[267] = False

## Step 3: Configure input stimuli into neurons
### Enter the normalized stimuli amplitude to the neurons to be stimulated. 
### In this tutorial, we inject constant input into PLML and PLMR

In [ ]:
input_vec = np.zeros(dw.network_sim.params_obj_neural['N'])

In [ ]:
# In this example, we are injecting 2nA constant current into PLMR (276th index) and PLML (278th index)

input_vec[276] = 2000
input_vec[278] = 2000

## Step 4: Run the simulation

In [ ]:
result_dict = dw.network_sim.run_network_constinput(t_duration=15, input_vec=input_vec, 
                                          ablation_mask=ablation_mask)

## Step 5: Extract the result

In [ ]:
result_dict.keys()

### In this example, we will take a look at the 'v_solution' 
### We take transpose to change the shape of the array to spatial neurons x temporal axis 

In [ ]:
v_sol = result_dict['v_solution'].T

### One can use matplotlib.pyplot.pcolor to visualize the activities of all neurons 
### Surprisingly, we see that the network doesn't oscillate anymore when PVC are ablated

In [ ]:
fig = plt.figure(figsize=(15, 8))
plt.pcolor(v_sol[:, 100:], cmap='bwr', vmin = -20, vmax = 20)
plt.colorbar()

### Or only the motorneuronal activities

In [ ]:
fig = plt.figure(figsize=(15, 8))
plt.pcolor(v_sol[dw.neurons_idx.motor_group, 100:], cmap='bwr', vmin = -20, vmax = 20)
plt.colorbar()

### Here we show the activities of Ventral Type B, D and Dorsal Type B, D neurons

In [ ]:
fig = plt.figure(figsize=(12, 8))
plt.subplot(4,1,1)
plt.pcolor(v_sol[dw.neurons_idx.VB_ind, 100:600], cmap='bwr')
plt.ylim(len(dw.neurons_idx.VB_ind), 0)
plt.colorbar()

fig = plt.figure(figsize=(12, 8))
plt.subplot(4,1,2)
plt.pcolor(v_sol[dw.neurons_idx.VD_ind, 100:600], cmap='bwr')
plt.ylim(len(dw.neurons_idx.VD_ind), 0)
plt.colorbar()

fig = plt.figure(figsize=(12, 8))
plt.subplot(4,1,3)
plt.pcolor(v_sol[dw.neurons_idx.DB_ind, 100:600], cmap='bwr')
plt.ylim(len(dw.neurons_idx.DB_ind), 0)
plt.colorbar()

fig = plt.figure(figsize=(12, 8))
plt.subplot(4,1,4)
plt.pcolor(v_sol[dw.neurons_idx.DD_ind, 100:600], cmap='bwr')
plt.ylim(len(dw.neurons_idx.DD_ind), 0)
plt.colorbar()